In [39]:
# installs
%pip install pyautogui
%pip install pyperclip
%pip install netifaces 
%pip installl scapy
%pip install pandas
%pip install tqdm
%pip install openpyx1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "installl" - maybe you meant "install"



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



ERROR: Could not find a version that satisfies the requirement openpyx1 (from versions: none)
ERROR: No matching distribution found for openpyx1


Note: you may need to restart the kernel to use updated packages.


In [40]:
# imports
import pyautogui
import pyperclip
import ipaddress
import multiprocessing
import netifaces as ni
from scapy.all import Ether, ARP, srp
from concurrent.futures import ThreadPoolExecutor
import datetime
import pandas as pd
import tqdm as progress_bar


In [41]:
interface = ni.gateways()['default'][ni.AF_INET][1]

# Obter o endereço IP e a máscara de rede da interface de rede
ip_addr = ni.ifaddresses(interface)[ni.AF_INET][0]['addr']
net_mask = ni.ifaddresses(interface)[ni.AF_INET][0]['netmask']

# Encontra todos os IPs no range da máscara de rede
network = ipaddress.IPv4Network(ip_addr + '/' + net_mask, strict=False)

ips_in_network = [str(ip) for ip in network.hosts()]

#ips_na_rede = ['192.168.1.1'] # todo: remover!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
print("IPs a serem verificados na rede: ", len(ips_in_network))
connected_devices = []

progress = progress_bar.tqdm(ips_in_network)

IPs a serem verificados na rede:  254


100%|██████████| 254/254 [17:13<00:00,  4.07s/it]


In [42]:
# funcoes
def ping_an_ip(ip):
    progress.update(1)
    answer = srp(Ether(dst="ff:ff:ff:ff:ff:ff")/ARP(pdst=ip), timeout=7, verbose=0)[0]
    if answer:
        connected_devices.append({'IP': answer[0][1].psrc, 'MAC': answer[0][1].hwsrc})

def open_gmail():
    #seleciona ícone do navegador, abre nova guia e acessa o gmail
    pyautogui.click(1023, 1053)
    pyautogui.hotkey('ctrl', 't')
    pyperclip.copy(text="https://mail.google.com/mail/u/0/?ogbl#inbox")
    pyautogui.hotkey('ctrl', 'v')
    pyautogui.PAUSE = 5
    pyautogui.press('enter')    
    pyautogui.PAUSE = 1

def create_email_message():
    #cria nova mensagem, adiciona email para envio e adiciona um título
    pyautogui.click(189, 192)
    pyperclip.copy(text="victorsoaresrj@hotmail.com")
    pyautogui.hotkey('ctrl', 'v')
    pyautogui.press('tab')
    pyperclip.copy(text="Informações atualizadas dos ips encontrados")
    pyautogui.hotkey('ctrl', 'v')

def attach_file_and_send_email():
    #abre pagina de anexo, navega até o arquivo, e sobe no corpo do email
    # abre opcao de anexo
    pyautogui.click(898, 984)

    # seleciona opcao de navegar até o arquivo
    pyautogui.click(665, 52)
    pyperclip.copy(text=r"C:\Users\JoãoVictorSoaresJord\Documents\ArquivoDaAutomação")
    pyautogui.hotkey('ctrl', 'v')
    pyautogui.press('enter')

    # seleciona arquivo
    pyautogui.click(402, 161)
    pyautogui.press('enter')

    # envia email
    pyautogui.click(750, 996)

In [43]:
cpu_count = multiprocessing.cpu_count()
print("Verificando dispositivos conectados na rede")
print("Número de núcleos disponíveis para a tarefa:", cpu_count)
with ThreadPoolExecutor(max_workers=cpu_count) as executor:
    executor.map(ping_an_ip, ips_in_network) # para cada ip em ips_na_rede, executa ping_an_ip
    executor.shutdown

for device in connected_devices:
    print("IP:", device['IP'], "| MAC:", device['MAC'])

Verificando dispositivos conectados na rede
Número de núcleos disponíveis para a tarefa: 16


 97%|█████████▋| 247/254 [01:45<00:03,  1.99it/s]

IP: 192.168.1.1 | MAC: e0:19:54:f6:5d:ed
IP: 192.168.1.100 | MAC: c6:39:be:b3:a5:dd
IP: 192.168.1.110 | MAC: 48:43:dd:86:26:e5
IP: 192.168.1.115 | MAC: 74:13:ea:82:2c:4c
IP: 192.168.1.105 | MAC: 78:dd:12:3c:76:5c


In [44]:
date_now = datetime.datetime.now()
formated_date_now = date_now.strftime("%Y-%m-%d%H-%M-%S")

df = pd.DataFrame(connected_devices, columns=['IP', 'MAC'])
complete_path_xlsx = f"C:\\Users\\JoãoVictorSoaresJord\\Documents\\ArquivoDaAutomação\\relatorio_rede_{formated_date_now}.xlsx"
df.to_excel(complete_path_xlsx, index=False)

In [46]:
open_gmail()
pyautogui.PAUSE = 1

create_email_message()
pyautogui.PAUSE = 1

attach_file_and_send_email()